In [131]:
import numpy as np

def sample_no_signalling(num: int=1)->np.array:
    A, B, C = np.random.uniform(size=(3,2,2))
    A = A/np.sum(A)
    B[0,:] = np.sum(A[0,:])*B[0,:]/np.sum(B[0,:])
    B[1,:] = np.sum(A[1,:])*B[1,:]/np.sum(B[1,:])
    C[:,0] = np.sum(B[:,0])*C[:,0]/np.sum(C[:,0])
    C[:,1] = np.sum(B[:,1])*C[:,1]/np.sum(C[:,1])
    min_sum = np.min(np.minimum(np.sum(A, axis=1), np.sum(C, axis=0)))
    d1 = np.random.uniform(0, min_sum)
    d2 = np.sum(C, axis=0)[0] - d1
    d3 = np.sum(A, axis=1)[1] - d2
    d4 = np.sum(A, axis=1)[0] - d1
    D = np.array([[d1, d2], [d3, d4]])
    result = np.concatenate((np.concatenate((A, B), axis=1), np.concatenate((D, C),axis=1)), axis=0)
    return result

In [132]:
def check_correctness(box: np.array)->bool:
    row_check = np.sum(box[:,:2], axis=1) == np.sum(box[:,2:], axis=1)
    col_check = np.sum(box[:2,:], axis=0) == np.sum(box[2:,:], axis=0)
    print(row_check, col_check)
    return np.all(row_check, col_check)

In [118]:
a = sample_noninformative()

In [ ]:
def check_locality(box: np.array)->bool:
    return True

In [ ]:
def generate_image(box: np.array)->np.array:
    return np.array()